In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode

In [2]:
data_sales = pd.read_csv('sales.csv',parse_dates=['date'])

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data_product_hierarchy = pd.read_csv('product_hierarchy.csv')

In [4]:
data_store_cities = pd.read_csv('store_cities.csv')

## Filling Nas for product hierarchy

In [6]:
data_product_hierarchy['product_length'].fillna(data_product_hierarchy['product_length'].mean(),inplace = True)
data_product_hierarchy['product_depth'].fillna(data_product_hierarchy['product_depth'].mean(),inplace = True)
data_product_hierarchy['product_width'].fillna(data_product_hierarchy['product_width'].mean(),inplace = True)
data_product_hierarchy['cluster_id'].fillna(mode(data_product_hierarchy['cluster_id'])[0][0],inplace = True)

In [7]:
data_product_hierarchy.isnull().sum()

product_id        0
product_length    0
product_depth     0
product_width     0
cluster_id        0
hierarchy1_id     0
hierarchy2_id     0
hierarchy3_id     0
hierarchy4_id     0
hierarchy5_id     0
dtype: int64

## Filling Nas for price

In [8]:
dict_price = {}
for i, x in enumerate(data_sales['product_id']):
    if (x not in dict_price.keys() and not(np.isnan(data_sales['price'][i]))):
        dict_price[x]=data_sales['price'][i]

In [9]:
data_sales.price = data_sales.price.fillna(data_sales.product_id.map(dict_price))

data_sales.isna().sum()

product_id                      0
store_id                        0
date                            0
sales                     1190762
revenue                   1190762
stock                     1190762
price                       10444
promo_type_1                    0
promo_bin_1              16803149
promo_type_2                    0
promo_bin_2              19395757
promo_discount_2         19395757
promo_discount_type_2    19395757
dtype: int64

In [10]:
data_sales['price'].fillna(data_sales['price'].mean(),inplace = True)

# Filling sales,revenue,stock and price by 

In [11]:
data_sales.isna().sum()

product_id                      0
store_id                        0
date                            0
sales                     1190762
revenue                   1190762
stock                     1190762
price                           0
promo_type_1                    0
promo_bin_1              16803149
promo_type_2                    0
promo_bin_2              19395757
promo_discount_2         19395757
promo_discount_type_2    19395757
dtype: int64

## Filling Nas for promos

In [12]:
data_sales['promo_bin_1'].fillna(0,inplace= True)
data_sales['promo_bin_2'].fillna(0,inplace= True)
data_sales['promo_discount_2'].fillna(0,inplace= True)
data_sales['promo_discount_type_2'].fillna(0,inplace= True)

In [13]:
data_sales.dropna(inplace=True)

In [14]:
data_sales.isna().sum()

product_id               0
store_id                 0
date                     0
sales                    0
revenue                  0
stock                    0
price                    0
promo_type_1             0
promo_bin_1              0
promo_type_2             0
promo_bin_2              0
promo_discount_2         0
promo_discount_type_2    0
dtype: int64

## Categorical to Numerical

#### promo_type_1

In [15]:
data_sales['promo_type_1'].unique()

array(['PR14', 'PR05', 'PR10', 'PR03', 'PR06', 'PR07', 'PR12', 'PR17',
       'PR08', 'PR13', 'PR09', 'PR16', 'PR15', 'PR01', 'PR18', 'PR11',
       'PR04'], dtype=object)

In [16]:
encode_dict_type_1 = {
    'PR01':1,
    'PR03':3,
    'PR04':4,
    'PR05':5,
    'PR06':6,
    'PR07':7,
    'PR08':8,
    'PR09':9,
    'PR10':10,
    'PR11':11,
    'PR12':12,
    'PR13':13,
    'PR14':14,
    'PR15':15,
    'PR16':16,
    'PR17':17,
    'PR18':18
}

data_sales['promo_type_1'].replace(encode_dict_type_1,inplace=True)

#### promo_bin_1

In [17]:
data_sales['promo_bin_1'].unique()

array([0, 'high', 'verylow', 'veryhigh', 'moderate', 'low'], dtype=object)

In [18]:
encode_dict = {
    'verylow':1,
    'low':2,
    'moderate':3,
    'high':4,
    'veryhigh':5
}

data_sales['promo_bin_1'].replace(encode_dict,inplace=True)

#### promo_type_2

In [19]:
data_sales['promo_type_2'].unique()

array(['PR03', 'PR02', 'PR01', 'PR04'], dtype=object)

In [20]:
encode_dict_type_2 = {'PR01':1,
                     'PR02':2,
                     'PR03':3,
                     'PR04':4}

data_sales['promo_type_2'].replace(encode_dict_type_2,inplace=True)

#### promo_bin_2

In [20]:
data_sales['promo_bin_2'].unique()

array([0, 'verylow', 'high', 'veryhigh'], dtype=object)

In [21]:
encode_dict_bin_2 = {
    'verylow':1,
    'low':2,
    'high':3,
    'veryhigh':4
}


data_sales['promo_bin_2'].replace(encode_dict_bin_2,inplace=True)

#### promo_discount_type_2

In [22]:
data_sales['promo_discount_type_2'].unique()

array([0, 'PR04', 'PR02', 'PR01', 'PR03'], dtype=object)

In [21]:
encode_discount_type_2 = {'PR01':1,
                     'PR02':2,
                     'PR03':3,
                     'PR04':4}

data_sales['promo_discount_type_2'].replace(encode_discount_type_2,inplace=True)

## Weekly Data Conversion

In [22]:
#Converting to weekly data

weekly_data = data_sales.groupby(['product_id','store_id']).resample('M', label='right', closed = 'right', on='date').sum().reset_index()
weekly_data

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_discount_2,promo_discount_type_2
0,P0001,S0001,2017-02-28,3.0,16.53,0.0,19.5,42,0,9,0.0,0
1,P0001,S0001,2017-03-31,0.0,0.00,0.0,0.0,0,0,0,0.0,0
2,P0001,S0001,2017-04-30,0.0,0.00,0.0,0.0,0,0,0,0.0,0
3,P0001,S0001,2017-05-31,0.0,0.00,0.0,0.0,0,0,0,0.0,0
4,P0001,S0001,2017-06-30,0.0,0.00,0.0,0.0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
696145,P0748,S0142,2019-06-30,0.0,0.00,156.0,245.7,182,0,39,0.0,0
696146,P0748,S0142,2019-07-31,0.0,0.00,432.0,585.9,416,8,93,0.0,0
696147,P0748,S0142,2019-08-31,3.0,45.50,714.0,585.9,416,8,93,0.0,0
696148,P0748,S0142,2019-09-30,3.0,45.50,580.0,567.0,402,8,90,0.0,0


In [23]:
sales_sorted = weekly_data.sort_values(by=['sales'],ascending=False)

sales_sorted

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_discount_2,promo_discount_type_2
85126,P0103,S0097,2017-02-28,83171.0,157983.652,1219.0,74.20,392,0,84,0.0,0
85128,P0103,S0097,2017-04-30,43753.0,85388.881,3374.0,84.85,420,0,90,0.0,0
366553,P0397,S0097,2017-06-30,6411.0,31183.060,100.0,345.10,406,0,87,0.0,0
418458,P0438,S0085,2017-03-31,3781.0,826.610,83449.0,7.75,434,0,93,0.0,0
418457,P0438,S0085,2017-02-28,3458.0,800.480,24519.0,7.00,392,0,84,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
501964,P0536,S0090,2018-03-31,0.0,0.000,341.0,116.25,434,0,93,0.0,0
219274,P0241,S0025,2018-11-30,0.0,0.000,60.0,508.50,294,14,90,0.0,0
219272,P0241,S0025,2018-09-30,0.0,0.000,90.0,433.50,420,0,90,0.0,0
219271,P0241,S0025,2018-08-31,0.0,0.000,106.0,405.45,434,0,93,0.0,0


In [24]:
sales_sorted = weekly_data.sort_values(by=['sales'],ascending=False)

sales_sorted.head(5)

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_discount_2,promo_discount_type_2
85126,P0103,S0097,2017-02-28,83171.0,157983.652,1219.0,74.20,392,0,84,0.0,0
85128,P0103,S0097,2017-04-30,43753.0,85388.881,3374.0,84.85,420,0,90,0.0,0
366553,P0397,S0097,2017-06-30,6411.0,31183.060,100.0,345.10,406,0,87,0.0,0
418458,P0438,S0085,2017-03-31,3781.0,826.610,83449.0,7.75,434,0,93,0.0,0
418457,P0438,S0085,2017-02-28,3458.0,800.480,24519.0,7.00,392,0,84,0.0,0


##### Product P0103 and Store S0097 have more sales in the given week 

In [25]:
store_high_sales = weekly_data.groupby(['store_id']).sum()

store_high_sales.sort_values(by=['sales'],ascending=False).head(5)

,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_discount_2,promo_discount_type_2
store_id,,,,,,,,,
S0085,432608.517,2156037.860,1.073722e+07,7.871690e+06,4182351,108621,975848,12456.0,969
S0097,322597.646,1488854.262,5.886279e+06,5.504291e+06,3588014,97047,836727,10941.0,875
S0026,218628.704,1317370.030,5.073888e+06,4.983100e+06,3427446,91329,798455,11114.0,882
S0062,208001.063,1299166.320,5.335818e+06,5.211048e+06,3446942,93092,803410,11843.0,926
S0038,203208.971,1027630.080,9.239523e+06,8.031128e+06,4294213,113146,1001872,11914.0,938


##### Store S0085 have more sales in the given Dataset

In [26]:
store_high_sales = weekly_data.groupby(['product_id']).sum()

store_high_sales.sort_values(by=['sales'],ascending=False).head(5)

,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_discount_2,promo_discount_type_2
product_id,,,,,,,,,
P0103,881634.0,2670199.663,10196627.0,448312.00,1568250,26912,383653,10480.0,2096
P0438,565348.0,217294.200,4197712.0,37501.30,1210146,0,259317,0.0,0
P0364,277763.0,312163.560,4493133.0,170230.85,1827984,4980,399390,16236.0,492
P0051,236744.0,175127.540,6003624.0,98746.30,1642997,20216,360204,0.0,0
P0388,215487.0,323230.500,186520.0,53523.00,499548,0,107046,0.0,0


##### Product P0103  have more sales in the Given dataset

In [27]:
# Encode Remaining Labels

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
weekly_data['product_id'] = le.fit_transform(weekly_data['product_id'])
weekly_data['store_id'] = le.fit_transform(weekly_data['store_id'])

In [29]:
sales_kmeans = weekly_data.drop(['revenue','stock','date','price','promo_type_1','promo_bin_1','promo_type_2','promo_discount_2','promo_discount_type_2'],axis = 1)
sales_kmeans.head(5)

,product_id,store_id,sales
0,0,0,3.0
1,0,0,0.0
2,0,0,0.0
3,0,0,0.0
4,0,0,0.0


# Cluster Analysis

In [33]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=5)

clusters = km.fit_predict(sales_kmeans)

In [ ]:
from sklearn.metrics import silhouette_score

score = silhouette_score(sales_kmeans, km.labels_, metric='euclidean')

print('Silhouetter Score: %.3f' % score)